# 3.1 Managed Services

# 3.2 Cloud Functions (GCP)

## Echo Service

In [1]:
!cat scripts/gcp_cloud_function/echo/requirements.txt

flask

In [3]:
!cat scripts/gcp_cloud_function/echo/main.py

def echo(request):
    import flask

    data = {"success": False}
    params = request.get_json()

    if "msg" in params:
        data["response"] = str(params['msg'])
        data["success"] = True
    
    return flask.jsonify(data)

In [6]:
URL_ECHO_SERVICE = "https://us-central1-semiotic-method-277520.cloudfunctions.net/echo"

In [7]:
import requests

result = requests.post(
    url=URL_ECHO_SERVICE, 
    json={'msg': 'Hello from Cloud Function'}
)
print(result.json())

{'response': 'Hello from Cloud Function', 'success': True}


## Cloud Storage (GCS)

In [44]:
from google.cloud import storage
bucket_name = "dsp_model_store_00"

storage_client = storage.Client()
storage_client.create_bucket(bucket_name)

for bucket in storage_client.list_buckets():
    print(bucket.name)

dsp_model_store_00


In [45]:
# save to GCS
from google.cloud import storage

bucket_name = "dsp_model_store_00"
storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

blob = bucket.blob("serverless/logit/v1")
blob.upload_from_filename("logit.pkl")

In [46]:
# load from GCS
import joblib
from google.cloud import storage

bucket_name = "dsp_model_store_00"
storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

blob = bucket.blob("serverless/logit/v1")
blob.download_to_filename("/tmp/local_logit.pkl")

model = joblib.load("/tmp/local_logit.pkl")
model

LogisticRegression()

## Model Function

In [16]:
!cat scripts/gcp_cloud_function/pred/requirements.txt

google-cloud-storage
sklearn
pandas
flask
joblib

In [17]:
!cat scripts/gcp_cloud_function/pred/main.py

def pred(request):

    import flask
    import joblib
    import sklearn
    import pandas as pd
    from google.cloud import storage

    data = {"success": False}
    params = request.get_json()
    
    if "G1" in params:
        new_row = {"G1": params.get("G1"), "G2": params.get("G2"),
                   "G3": params.get("G3"), "G4": params.get("G4"),
                   "G5": params.get("G5"), "G6": params.get("G6"),
                   "G7": params.get("G7"), "G8": params.get("G8"),
                   "G9": params.get("G9"), "G10":params.get("G10")}
        
        new_x = pd.DataFrame.from_dict(data=new_row, 
                                       orient="index",
                                       dtype="float").T

        # set up access to the GCS bucket
        bucket_name = "dsp_model_store_00"
        storage_client = storage.Client()
        bucket = storage_client.get_bucket(bucket_name)

        # download and load the model
        blob = bucket.blob("serverless/lo

In [18]:
URL_PRED_SERVICE = "https://us-central1-semiotic-method-277520.cloudfunctions.net/pred"

In [36]:
import requests

result = requests.post(
    url=URL_PRED_SERVICE,
    json={'G1': '1', 'G2': '0', 'G3': '0', 'G4': '0', 'G5': '0',
          'G6': '0', 'G7': '0', 'G8': '0', 'G9': '0', 'G10': '0'})
print(result.json())

{'response': '0.06730006696024816', 'success': True}


In [40]:
%%timeit
result = requests.post(
    url=URL_PRED_SERVICE,
    json={'G1': '1', 'G2': '0', 'G3': '0', 'G4': '0', 'G5': '0',
          'G6': '0', 'G7': '0', 'G8': '0', 'G9': '0', 'G10': '0'})

337 ms ± 28.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
!cat scripts/gcp_cloud_function/pred/main_fast.py

model = None

def pred_fast(request):
    global model

    import flask
    import joblib
    import sklearn
    import pandas as pd
    from google.cloud import storage

    data = {"success": False}
    params = request.get_json()
    
    if "G1" in params:
        new_row = {"G1": params.get("G1"), "G2": params.get("G2"),
                   "G3": params.get("G3"), "G4": params.get("G4"),
                   "G5": params.get("G5"), "G6": params.get("G6"),
                   "G7": params.get("G7"), "G8": params.get("G8"),
                   "G9": params.get("G9"), "G10":params.get("G10")}
        
        new_x = pd.DataFrame.from_dict(data=new_row, 
                                       orient="index",
                                       dtype="float").T

        if not model:
            # set up access to the GCS bucket
            bucket_name = "dsp_model_store_00"
            storage_client = storage.Client()
            bucket = storage_client.get_bucket(bucket_name)

     

In [38]:
URL_PRED_FAST_SERVICE = "https://us-central1-semiotic-method-277520.cloudfunctions.net/pred_fast"

In [39]:
import requests

result = requests.post(
    url=URL_PRED_FAST_SERVICE,
    json={'G1': '1', 'G2': '0', 'G3': '0', 'G4': '0', 'G5': '0',
          'G6': '0', 'G7': '0', 'G8': '0', 'G9': '0', 'G10': '0'})
print(result.json())

{'response': '0.06730006696024816', 'success': True}


In [41]:
%%timeit
result = requests.post(
    url=URL_PRED_FAST_SERVICE,
    json={'G1': '1', 'G2': '0', 'G3': '0', 'G4': '0', 'G5': '0',
          'G6': '0', 'G7': '0', 'G8': '0', 'G9': '0', 'G10': '0'})

105 ms ± 4.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [2]:


# CLOUD STORAGE
# https://cloud.google.com/appengine/docs/standard/python/googlecloudstorageclient/read-write-to-cloud-storage

#import logging
#import os
#import cloudstorage as gcs
#import webapp2

#from google.appengine.api import app_identity


ModuleNotFoundError: No module named 'google.appengine'

In [7]:

import requests

result = requests.post("https://3z5btf0ucb.execute-api.us-east-1.amazonaws.com/default/logit", \
                       json = { 'G1': '1', 'G2': '0', 'G3': '0', 'G4': '0', 'G5': '0', 'G6': '0', 'G7': '0', 'G8': '0', 'G9': '0', 'G10': '0' })
print(result.text)





Prediction 0.06745113592634559


In [28]:
from google.cloud import storage

storage_client = storage.Client()
bucket_name = "dsp_model_store_1"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob("serverless/logit/v1")

contents = blob.download_as_string()
contents

b'\x80\x03csklearn.linear_model.logistic\nLogisticRegression\nq\x00)\x81q\x01}q\x02(X\x07\x00\x00\x00penaltyq\x03X\x02\x00\x00\x00l2q\x04X\x04\x00\x00\x00dualq\x05\x89X\x03\x00\x00\x00tolq\x06G?\x1a6\xe2\xeb\x1cC-X\x01\x00\x00\x00Cq\x07G?\xf0\x00\x00\x00\x00\x00\x00X\r\x00\x00\x00fit_interceptq\x08\x88X\x11\x00\x00\x00intercept_scalingq\tK\x01X\x0c\x00\x00\x00class_weightq\nNX\x0c\x00\x00\x00random_stateq\x0bNX\x06\x00\x00\x00solverq\x0cX\x04\x00\x00\x00warnq\rX\x08\x00\x00\x00max_iterq\x0eKdX\x0b\x00\x00\x00multi_classq\x0fh\rX\x07\x00\x00\x00verboseq\x10K\x00X\n\x00\x00\x00warm_startq\x11\x89X\x06\x00\x00\x00n_jobsq\x12NX\x08\x00\x00\x00l1_ratioq\x13NX\x08\x00\x00\x00classes_q\x14cnumpy.core.multiarray\n_reconstruct\nq\x15cnumpy\nndarray\nq\x16K\x00\x85q\x17C\x01bq\x18\x87q\x19Rq\x1a(K\x01K\x02\x85q\x1bcnumpy\ndtype\nq\x1cX\x02\x00\x00\x00i8q\x1dK\x00K\x01\x87q\x1eRq\x1f(K\x03X\x01\x00\x00\x00<q NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq!b\x89C\x10\x00\x00\x00\x00\x00\x00\x00\x00\x

In [24]:
blob

<Blob: dsp_model_store_1, serverless/logit/v1, None>

In [ ]:
from google.cloud import storage

storage_client = storage.Client()

def hello_gcs_generic(data, context):
    bucket = storage_client.get_bucket(data['bucket'])
    blob = bucket.blob(data['name'])
    contents = blob.download_as_string()
    # Process the file contents, etc...

In [10]:
import cloudstorage as gcs

In [11]:
filenamee = "gs://dsp_model_store_1/serverless/logit/v1/local_logit.pkl"

gcs_file = gcs.open(filename)
contents = gcs_file.read()
gcs_file.close()

AttributeError: module 'cloudstorage' has no attribute 'open'

In [ ]:
import pickle 

blob = bucket.blob("serverless/logit/v1")
#blob.download_to_filename("local_logit.pkl")
#model = pickle.load(open("local_logit.pkl", 'rb'))

sm = blob.download_as_string("local_logit.pkl")
#model = pickle.load(sm)


#model


In [2]:
from google.resumable_media.requests import Download



In [ ]:

download = Download(
                download_url, stream=file_obj, headers=headers, start=start, end=end
            )
            download.consume(transport)

In [30]:
import pickle 

blob = bucket.blob("serverless/logit/v1")
#blob.download_to_filename("local_logit.pkl")
#model = pickle.load(open("local_logit.pkl", 'rb'))

sm = blob.download_as_string("local_logit.pkl")
#model = pickle.load(sm)


#model


AttributeError: 'str' object has no attribute '_http'

In [25]:
blob = bucket.blob("serverless/logit/v1")
blob

<Blob: dsp_model_store_1, serverless/logit/v1, None>

In [ ]:
storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(source_blob_name)

    blob.download_to_filename(destination_file_name)

    print('Blob {} downloaded to {}.'.format(
        source_blob_name,
        destination_file_name))